# Data Cleaning Overview
This notebook's purpose is to perform the dataset cleaning steps including those detailed in the Exploration notebook, but also any additional needed as further exploration is done.

While the visualization of the data in this notebook is interesting to the analyzer, a production process for cleaning the data should live in a script instead. This notebook is showing a proof of concept of data cleaning.


## Expected Cleaning Steps
- Remove `url`, `email_link`, and `date_of_collection` columns
- Separate `rfs` columns into granularity aspects (year, month/quarter) and cast to appropriate types
- Due to many to many relationship, separate owners and landing_point information into separate tablelike datasets.
- Cast `length` to numeric and add unit information to column name
- Fill `is_tbd` nulls with False
- Collect and append latitude/longitude data for landing points
- Potentially feature engineer to get landing point city

## Actual Cleaning Steps
- Remove `url`, `email_link`, and `date_of_collection` columns
- Cast `length` to numeric and add unit information to column name
- Separate `rfs` columns into granularity aspects (year, month/quarter) and cast to appropriate types
- Drop `rfs_sub_year` columns due to the significant number of null values. In the future, may still be useful
- Due to many to many relationship, separate owners and landing_point information into separate tablelike datasets.
- feature engineer to get landing point sub country granularity from `name` field by removing country info
- Fill `is_tbd` nulls with False
- Collect and create separate latitude/longitude data for landing points
    - Ended up making it a separate dataset to cut down on data duplication
    - Required several iterations and extensive exploration. See below for further information.
- Save datasets locally and then upload to S3
    - Done because this is a one-time thing. If process runs multiple times, should code out programatic upload.

## Lat/Long Coordinate Append Steps
In order to visualize the location of landing points on a map, coordinates in latitude and longitude are required but are not provided by the dataset. To collect this information, I attempted several different methods. In the end, a mixture of methods was used.

For landing site granularity, so that different landing points within the same country are separated visually, I cleaned the `name` field to attempt to pull out sub country information including city and region. However, this level of specificity proved difficult to collect accurate coordinates for due to API limits and missing address information (ie, San Jose in Phillipines. There are 9 different cities in the phillipines with that city name.) So, along with more specific coordinates, I also included overall country coordinates in the final output.

Data Issues:
- Country name not consistent across datasets, required cleaning
- City name not consistent across datasets (punctuation), was not cleaned due to sheer number

Methods considered:
1. Find public dataset of city coordinates
    - I used https://simplemaps.com/data/world-cities, which is free-use so long as credit is given.
    - Also explored https://www.geonames.org/maps/addresses.html#geoCodeAddress, but this is missing coordinates for most countries.
    - Issues:
        - Not all cities are listed resulted in many missing datapoints
        - Only half of landing point locations are able to be matched using this dataset
    - Next Steps Options:
        - Use Levenstein distance to calculate a fuzzy match on city. Did not pursue now because this could easily match to incorrect city if off by a single letter.
2. Use a free API to pass address info and get back coordinates.
    - Because the dataset is so small, maybe we can use an API to collect info without hitting rate limits
    - Used https://www.gpsvisualizer.com/geocoder/ with MapQuest application token.
    - Issues:
        - Slow, returned one result every second
        - Doesn't seem to be able to handle city, country combos. Had lots of incorrect mappings (ie, China city mapped to Washington DC USA coordinates).
3. Find a public dataset of overall country coordinates.
    - Abandoning granularity, at least append the overall country coordinates.
    - Here, I used the Kaggle dataset https://www.kaggle.com/datasets/paultimothymooney/latitude-and-longitude-for-every-country-and-state which also included US state coordinates.
    - Issues:
        - Missing a couple countries in the submarine dataset. Needed to be appended using hardcoding.

Approach: 
- Have two columns that have the most granularity available (`latitude` and `longitude`) and two columns that simply have the overall country coordinates (`latitude_country` and `longitude_country`).
- The lowest granularity for a landing point was found by passing the location data through several matching processes that became less and less granular. They are as follows:
    - Match Level 1: City, Region, and Country
    - Match Level 2: City and Country
    - Match Level 3 (USA Only): USA State and Country
    - Match Level 4: Country
    - Match Level 5: Hardcoded Country
    

# Imports and Raw Cable Data Load

In [1]:
# Imports
import json
from pathlib import Path

import pandas as pd

from us_state_abbr import abbrev_to_us_state

# Constants
RAW_DATA_PATH = Path("raw_data")
CLEAN_DATA_PATH = Path("cleaned_data")

RAW_CABLE_DATA_PATH = RAW_DATA_PATH / "Submarine Cables - 2023-02-22.json"

In [4]:
# Load sub cable datasets
with RAW_CABLE_DATA_PATH.open("r") as data_file:
    raw_cable_data = json.load(data_file)

# Switch from json to dataframe for easier manipulation
raw_cable_df = pd.DataFrame.from_dict(raw_cable_data)

# Create new object to represent manipulated data
clean_cable_df = raw_cable_df.copy()

In [5]:
# CLEANING: Remove `url`, `email_link`, and `date_of_collection` columns
clean_cable_df = clean_cable_df.drop(columns=['date_of_collection', 'url', 'email_link'])


# CLEANING: Cast `length` to numeric and add unit information to column name
clean_cable_df['length'] = clean_cable_df['length'].str[:-3].str.replace(',', '').astype('Int64')
clean_cable_df = clean_cable_df.rename(columns={'length': 'length (km)'})


# CLEANING: Separate `rfs` columns into granularity aspects (year, month/quarter) and cast to appropriate types
#     Remove raw rfs column
rfs_split = clean_cable_df['rfs'].str.split()
assert rfs_split.str.len().max() <= 2 # make sure there are at max two data points
clean_cable_df['rfs_year'] = rfs_split.str[0].astype('Int16') # Int16 to handle nulls
clean_cable_df['rfs_sub_year'] = rfs_split.str[1]
clean_cable_df = clean_cable_df.drop(columns=['rfs'])

clean_cable_df.head()

,cable_id,cable_name,owners,landing_points,length (km),rfs_year,rfs_sub_year
0,2africa,2Africa,"China Mobile, MTN, Meta, Orange, Saudi Telecom...","[{""is_tbd"": null, ""country"": ""Angola"", ""id"": ""...",45000,2023,NaN
1,acs-alaska-oregon-network-akorn,ACS Alaska-Oregon Network (AKORN),Alaska Communications,"[{""is_tbd"": null, ""country"": ""United States"", ...",3000,2009,April
2,aden-djibouti,Aden-Djibouti,"Djibouti Telecom, Orange, Tata Communications,...","[{""is_tbd"": null, ""country"": ""Djibouti"", ""id"":...",269,1994,NaN
3,adria-1,Adria-1,"ALBtelecom, Hrvatski Telekom","[{""is_tbd"": null, ""country"": ""Albania"", ""id"": ...",440,1996,September
4,aec-1,AEC-1,Aqua Comms,"[{""is_tbd"": null, ""country"": ""Ireland"", ""id"": ...",5521,2016,January


In [6]:
# Explore cleaned columns again

print("Distribution of the rfs year:\n", clean_cable_df['rfs_year'].describe())

print("\nDistribution of the length:\n", clean_cable_df['length (km)'].describe())

# Don't want to make the assumption that Q1 means beginning of year as they may be business quarters
# FINDING: Given large number of nulls and mismatch of month vs quarter, dropping this col for now
#     may still be of use when additional data can be provided
print("\nValue counts of the rfs sub year unit:\n", clean_cable_df['rfs_sub_year'].value_counts(dropna=False))

Distribution of the rfs year:
 count          540.0
mean     2010.298148
std        10.063343
min           1989.0
25%           2001.0
50%           2011.0
75%           2019.0
max           2026.0
Name: rfs_year, dtype: Float64

Distribution of the length:
 count          508.0
mean     3425.437008
std      6001.366601
min              5.0
25%           228.75
50%            816.0
75%           3478.5
max          45000.0
Name: length (km), dtype: Float64

Value counts of the rfs sub year unit:
 NaN          172
December      47
June          32
September     29
January       29
March         29
July          28
August        24
November      24
October       23
February      21
April         20
May           18
Q1            14
Q4            12
Q3             9
Q2             9
None           5
Name: rfs_sub_year, dtype: int64


In [7]:
# CLEANING: Drop `rfs_sub_year` columns due to the significant number of null values. In the future, may still be useful
clean_cable_df.drop(columns=['rfs_sub_year'], inplace=True)

# Split into three datasets
Cable, Owner, Landing Point

In [13]:
# Because cable_id is unique, there will be no duplicate rows in the below datasets
cable_data = clean_cable_df[["cable_id", "cable_name", "length (km)", "rfs_year"]].copy()
owner_data = clean_cable_df[['owners', 'cable_id']].copy()
location_data = clean_cable_df[['landing_points', 'cable_id']].copy()

In [155]:
owner_data['owners'] = owner_data['owners'].str.split(', ')
owner_data = owner_data.explode('owners', ignore_index=True).rename(columns={'owners': 'owner'})
owner_data.head()

,owner,cable_id
0,China Mobile,2africa
1,MTN,2africa
2,Meta,2africa
3,Orange,2africa
4,Saudi Telecom,2africa


In [14]:
# Countries are in alphabetical order, so probably not in cable order. Don't need to track order
# lambda is slow here, but dataset is small. If dataset scales, will need a new solution
location_data['landing_points'] = location_data['landing_points'].apply(lambda x: json.loads(x))
location_data = location_data.explode('landing_points', ignore_index=True)
location_data = pd.concat([location_data['landing_points'].apply(pd.Series), location_data['cable_id']], axis=1)

# CLEANING: feature engineer to get landing point city and region granularity from `name` field 
#     by removing country info
# Remove `name` column after as it contains no additional information
location_data['sub_country'] = location_data.apply(lambda x: x['name'].replace(", " + x['country'], ""), axis=1)
loc_split = location_data['sub_country'].str.split(", ")
location_data['region'] = loc_split.str[1]
location_data['city'] = loc_split.str[0]
location_data.drop(columns=['name', 'sub_country'], inplace=True)


# CLEANING: Fill `is_tbd` nulls with False
location_data['is_tbd'] = location_data['is_tbd'].fillna(False)


# Visualize
print("Column types and nulls:\n", location_data.info())
location_data.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2578 entries, 0 to 2577
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   is_tbd    2578 non-null   bool  
 1   country   2578 non-null   object
 2   id        2578 non-null   object
 3   cable_id  2578 non-null   object
 4   region    351 non-null    object
 5   city      2578 non-null   object
dtypes: bool(1), object(5)
memory usage: 103.3+ KB
Column types and nulls:
 None


,is_tbd,country,id,cable_id,region,city
0,False,Angola,luanda-angola,2africa,NaN,Luanda
1,False,Bahrain,manama-bahrain,2africa,NaN,Manama
2,False,Comoros,moroni-comoros,2africa,NaN,Moroni


In [20]:
# Create latitude / longitude dataset

# Saved dataset of world city latitude/longitude
RAW_LAT_LONG_CITY_PATH = RAW_DATA_PATH / "worldcities.csv"
RAW_LAT_LONG_COUNTRY_HARDCODED_PATH = RAW_DATA_PATH / "country_lat_long_hard_coded.csv"
RAW_LAT_LONG_US_STATE_AND_COUNTRY_PATH = RAW_DATA_PATH / "country_and_usa_states_lat_long.csv"


# Hard code some country name translations because no 2-letter country code is provided in submarine dataset
country_cleaning = {
    "British Virgin Islands": "Virgin Islands (U.K.)",
    "U.S. Virgin Islands": "Virgin Islands (U.S.)",
    "Congo [DRC]": "Congo, Dem. Rep.",
    "Congo [Republic]": "Congo, Rep.",
    "Myanmar [Burma]": "Myanmar",
    "Saint Helena": "Saint Helena, Ascension and Tristan da Cunha",
    "Cocos [Keeling] Islands": "Cocos (Keeling) Islands",
}


# Match 1 Dataset: Dataset for match city, admin, country
lat_long_city_admin_country = (
    pd.read_csv(RAW_LAT_LONG_CITY_PATH)[['city_ascii', 'admin_name', 'country', 'lat', 'lng']]
    .rename(columns={'city_ascii': 'city', 'lat': 'latitude', 'lng': 'longitude', 'admin_name': 'region'})
)
lat_long_city_admin_country['country'] = (
    lat_long_city_admin_country['country']
    .apply(lambda x: country_cleaning.get(x, x))
)


# Match 2 Dataset: Dataset for match city, country
lat_long_city_country = lat_long_city_admin_country.drop(columns=['region'])
unique_combos = lat_long_city_country.groupby(['country', 'city']).nunique().reset_index()
unique_combos = unique_combos[(unique_combos['latitude'] == 1) & (unique_combos['longitude'] == 1)]
lat_long_city_country = (
    lat_long_city_country
    .merge(unique_combos[['country', 'city']], on=['country', 'city'], how='inner')
)


# Match 3 Dataset: Dataset for match usa state
lat_long_country_state = pd.read_csv(RAW_LAT_LONG_US_STATE_AND_COUNTRY_PATH)
lat_long_country_state['country'] = lat_long_country_state['country'].apply(lambda x: country_cleaning.get(x, x))

lat_long_usa_state = (
    lat_long_country_state[['usa_state', 'country', 'usa_state_latitude', 'usa_state_longitude']]
    .copy()
    .dropna()
    .rename(columns={'usa_state_latitude': 'latitude', 'usa_state_longitude': 'longitude', 'usa_state': 'region'})
)
lat_long_usa_state['country'] = 'United States'


# Match 4 Dataset: Dataset for match country
lat_long_country = lat_long_country_state[['country', 'latitude', 'longitude']].copy()


# Match 5 Dataset: Hardcoded for match country
lat_long_country_hardcoded = pd.read_csv(RAW_LAT_LONG_COUNTRY_HARDCODED_PATH)


In [28]:
# Drop duplicates because a single landing point will be listed multiple times if connected to multiple `cable_id`s
unique_location_df = location_data[['id', 'country', 'city', 'region']].drop_duplicates()

# CLEANING: US State abbreviations to full name
#     US states are set to abbreviations in submarine which is great, but lat/long datasets have full name
def clean_us_state(df_row):
    if df_row['country'] == 'United States':
        return abbrev_to_us_state.get(df_row['region'])
    return df_row['region']

unique_location_df['region'] = unique_location_df.apply(lambda x:  clean_us_state(x), axis=1)


### Do Matching
match_1 = (
    unique_location_df[unique_location_df['region'].notnull()]
    .merge(
        lat_long_city_admin_country, 
        how='inner', 
        on=['country', 'city', 'region'])
)
match_1['match'] = "1_city_region_country"
remaining = unique_location_df[~unique_location_df['id'].isin(match_1['id'])]
print(f"\nmatch 1 percentage: {match_1.shape[0] / unique_location_df.shape[0]:.4f}, count: {match_1.shape[0]}")
print(f"remaining percentage: {remaining.shape[0]/ unique_location_df.shape[0]:.4f}, count: {remaining.shape[0]}")


match_2 = (
    remaining
    .merge(
        lat_long_city_country, 
        how='inner', 
        on=['country', 'city'])
)
match_2['match'] = "2_city_country"
remaining = remaining[~remaining['id'].isin(match_2['id'])]
print(f"\nmatch 2 percentage: {match_2.shape[0] / unique_location_df.shape[0]:.4f}, count: {match_2.shape[0]}")
print(f"remaining percentage: {remaining.shape[0]/ unique_location_df.shape[0]:.4f}, count: {remaining.shape[0]}")


match_3 = (
    remaining
    .merge(lat_long_usa_state, how='inner', on=['country', 'region'])
)
match_3['match'] = "3_usa_state"
remaining = remaining[~remaining['id'].isin(match_3['id'])]
print(f"\nmatch 3 percentage: {match_3.shape[0] / unique_location_df.shape[0]:.4f}, count: {match_3.shape[0]}")
print(f"remaining percentage: {remaining.shape[0]/ unique_location_df.shape[0]:.4f}, count: {remaining.shape[0]}")


match_4 = (
    remaining
    .merge(lat_long_country, how='inner', on=['country'])
)
match_4['match'] = "4_country"
remaining = remaining[~remaining['id'].isin(match_4['id'])]
print(f"\nmatch 4 percentage: {match_4.shape[0] / unique_location_df.shape[0]:.4f}, count: {match_4.shape[0]}")
print(f"remaining percentage: {remaining.shape[0]/ unique_location_df.shape[0]:.4f}, count: {remaining.shape[0]}")


match_5 = (
    remaining
    .merge(lat_long_country_hardcoded, how='inner', on=['country'])
)
match_5['match'] = "5_country_hardcoded"
remaining = remaining[~remaining['id'].isin(match_5['id'])]
print(f"\nmatch 4 percentage: {match_5.shape[0] / unique_location_df.shape[0]:.4f}, count: {match_5.shape[0]}")
print(f"remaining percentage: {remaining.shape[0]/ unique_location_df.shape[0]:.4f}, count: {remaining.shape[0]}")

location_with_lat_long = pd.concat(
    [match_1, match_2, match_3, match_4, match_5],
    axis=0
)
assert location_with_lat_long.shape[0] == unique_location_df.shape[0]

location_with_lat_long.head()


match 1 percentage: 0.0249, count: 39
remaining percentage: 0.9751, count: 1525

match 2 percentage: 0.3242, count: 507
remaining percentage: 0.6509, count: 1018

match 3 percentage: 0.0486, count: 76
remaining percentage: 0.6023, count: 942

match 4 percentage: 0.5953, count: 931
remaining percentage: 0.0070, count: 11

match 4 percentage: 0.0070, count: 11
remaining percentage: 0.0000, count: 0


,id,country,city,region,latitude,longitude,match
0,anchorage-ak-united-states,United States,Anchorage,Alaska,61.1508,-149.1091,1_city_region_country
1,florence-or-united-states,United States,Florence,Oregon,43.9916,-124.1063,1_city_region_country
2,shirley-ny-united-states,United States,Shirley,New York,40.7936,-72.8748,1_city_region_country
3,juneau-ak-united-states,United States,Juneau,Alaska,58.4546,-134.1739,1_city_region_country
4,lynnwood-wa-united-states,United States,Lynnwood,Washington,47.8284,-122.3033,1_city_region_country


In [29]:
# Add country granularity as well for a consistent view
all_country_lat_long = (
    pd.concat([lat_long_country_hardcoded, lat_long_country], axis=0)
    .rename(columns={'latitude': 'latitide_country', 'longitude': 'longitude_country'})
)
location_with_lat_long_country = (
    location_with_lat_long
    .merge(all_country_lat_long, on='country', how='inner')
)
assert location_with_lat_long_country.shape[0] == location_with_lat_long.shape[0]

# Write Datasets

In [163]:
folder = Path("cleaned_data")
folder.mkdir(exist_ok=True)

cable_data.to_csv(folder / "cable.csv")
owner_data.to_csv(folder / "owner.csv")
location_data.to_csv(folder / "location.csv")
location_with_lat_long_country.to_csv(folder / "location_lat_long.csv")